In [ ]:
!pip -q install alibi

In [ ]:
import alibi
import numpy as np
import torch

In [ ]:
import torch
import torch.optim as optim
from torch.optim import lr_scheduler

import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

import torchvision
from torchvision import datasets, models, transforms
import copy

import numpy as np
import pandas as pd

%matplotlib inline 
import matplotlib.pyplot as plt
import time
import os
import copy
import random
import math
import string

import tqdm

import scipy.stats as stats

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture

from skimage.filters import sobel
from skimage.color import rgb2gray

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [ ]:
#This is the classifier Class.
from torchvision.transforms.transforms import RandomRotation, RandomAdjustSharpness, RandomGrayscale

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = False, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone # Bool: Indicates if we use multibackbone

    #In the following secttion we download the appropriatee prettrained model
    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.multi_backbone: #We create the backbones and put them on the device
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)

    else:
      self.backbone =  nn.Sequential(*modules).to(device)


    #This is the final classification layer
    self.fc = nn.Sequential(nn.Dropout(dropout_rate),
                            nn.Linear(self.out_channels * 4, 3))                
     
  def forward(self, x, is_training = True):
    # Transform image range from (0, 1) to (-1, 1)

    imgs = [x[:,i] for i in range(4)] #list of 4 images
    

    if self.multi_backbone:
      encodings = [self.backbone1(imgs[0]).flatten(1), 
                   self.backbone2(imgs[1]).flatten(1),
                   self.backbone3(imgs[2]).flatten(1),
                   self.backbone4(imgs[3]).flatten(1)]
    else:
      encodings = [self.backbone(img).flatten(1) for img in imgs]

    return self.fc(torch.cat(encodings,1))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instantiate the classifier using the following weights
# If your directory structure does not resemble the following, you may need to 
model = torch.load("drive/MyDrive/Fish Attribution/" + 'model1e-050.5.2022-05-11 13_52_21.pt', map_location="cpu")
model.eval()
model.zero_grad()

In [ ]:
def predictor(X: np.ndarray) -> np.ndarray:
    X = torch.as_tensor(X, device='cpu')
    return model.forward(X).detach().numpy()

In [ ]:
# Load the raw fish images
# WARNING: The following WILL use a large amount of RAM
import numpy as np

X_PATH = "/content/drive/Shareddrives/Exploding Gradients/X_cropped_b.npy"
Y_PATH= "/content/drive/Shareddrives/Exploding Gradients/y_b.npy"

x = np.load(X_PATH)
y = np.load(Y_PATH)

# 285 instances, currently the shapes are off, see next cell for fix
print(x.shape)
print(y.shape)

(285, 4, 130, 750, 3)
(285, 1)


In [ ]:
# Conver to PyTorch tensors from raw numpy arrays and
# fix the dimensions
tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y).long()

tensor_x = torch.swapaxes(tensor_x,2,4)
tensor_x = torch.swapaxes(tensor_x,3,4)

In [ ]:
# Create a PyTorch dataloader to feed data to the model
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

attribution_ds = TensorDataset(tensor_x ,tensor_y) 

# Each time this is called, 40 pieces of data (40x4 = 160) are provided
# This was figured out in a bit of trial-and-error fashion because
# you have a limited about of onboard GPU RAM. 40 seems to be the maximum
# it's happy with and any Colab Pro instance goes into the red at this point
attribution_dl = DataLoader(attribution_ds,1,shuffle = True)

# Delete the raw numpy arrays from RAM, there's no need for them
# now that they exist as PyTorch tensors
del x,y

In [ ]:
image, label = iter(attribution_dl).next()

In [ ]:
np.array(image.shape)

array([  1,   4,   3, 130, 750])

In [ ]:
predictor(image)

array([[ 0.19444455,  0.46261984, -0.749903  ]], dtype=float32)

In [ ]:
mask = np.load("drive/MyDrive/Fish Attribution" + "/13x25 Mask.npy")

In [ ]:
def segmenter(image):
  return mask

In [ ]:
from alibi.explainers import AnchorImage

explainer = AnchorImage(predictor, np.array(image.squeeze().shape), segmentation_fn=segmenter,
                        images_background=None)

In [ ]:
explanation = explainer.explain(image.squeeze().detach().numpy(), threshold=.95, p_sample=.8, seed=0)

IndexError: ignored

In [ ]:
mask.shape

(4, 3, 130, 750)

In [ ]:
image.shape

torch.Size([1, 4, 3, 130, 750])